In [88]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statistics
import time

# to ignore the warnings
from warnings import filterwarnings

# Data From API

In [89]:
scores_df = pd.read_excel('./data/game_scores.xlsx')
scores_df.head()

,date,visitor,home,visitor_points,home_points
0,2015-11-25T01:00:00.000Z,Boston Celtics,Atlanta Hawks,97.0,121.0
1,2016-04-09T23:30:00.000Z,Boston Celtics,Atlanta Hawks,107.0,118.0
2,2016-04-16T23:00:00.000Z,Boston Celtics,Atlanta Hawks,101.0,102.0
3,2016-04-19T23:00:00.000Z,Boston Celtics,Atlanta Hawks,72.0,89.0
4,2016-04-27T00:30:00.000Z,Boston Celtics,Atlanta Hawks,83.0,110.0


In [90]:
scores_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14242 entries, 0 to 14241
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            14242 non-null  object 
 1   visitor         14242 non-null  object 
 2   home            14242 non-null  object 
 3   visitor_points  12796 non-null  float64
 4   home_points     12796 non-null  float64
dtypes: float64(2), object(3)
memory usage: 556.5+ KB


# Date Correction

In [91]:
scores_df.loc[scores_df['date'] <= '2024-10-06']

,date,visitor,home,visitor_points,home_points
0,2015-11-25T01:00:00.000Z,Boston Celtics,Atlanta Hawks,97.0,121.0
1,2016-04-09T23:30:00.000Z,Boston Celtics,Atlanta Hawks,107.0,118.0
2,2016-04-16T23:00:00.000Z,Boston Celtics,Atlanta Hawks,101.0,102.0
3,2016-04-19T23:00:00.000Z,Boston Celtics,Atlanta Hawks,72.0,89.0
4,2016-04-27T00:30:00.000Z,Boston Celtics,Atlanta Hawks,83.0,110.0
...,...,...,...,...,...
14236,2020-02-29T02:00:00.000Z,Washington Wizards,Utah Jazz,119.0,129.0
14237,2021-04-13T01:00:00.000Z,Washington Wizards,Utah Jazz,125.0,121.0
14238,2021-12-19T02:00:00.000Z,Washington Wizards,Utah Jazz,109.0,103.0
14239,2022-12-23T02:00:00.000Z,Washington Wizards,Utah Jazz,112.0,120.0


In [92]:
# Find columns and total null values
scores_df.isnull().sum()

date                 0
visitor              0
home                 0
visitor_points    1446
home_points       1446
dtype: int64

In [93]:
# From this dataframe it is found that games where the dates are in the morning are previous day games
# These games will have to be adjusted to match actual game schedules 

# 1. Convert date column to datetime
scores_df['date'] = pd.to_datetime(scores_df['date'], errors='coerce')

# 2. Find days between 12 am and 3 am and subtract one day for each row (row count started to change from 3am to 4am, 3am was the limit)
scores_df.loc[scores_df['date'].dt.hour.between(0, 3), 'date'] -= pd.Timedelta(days=1)

# 3.Format the 'date' column to show only the date portion (year-month-day)
scores_df['date'] = scores_df['date'].dt.strftime('%Y-%m-%d')

# Display
scores_df.head()

,date,visitor,home,visitor_points,home_points
0,2015-11-24,Boston Celtics,Atlanta Hawks,97.0,121.0
1,2016-04-09,Boston Celtics,Atlanta Hawks,107.0,118.0
2,2016-04-16,Boston Celtics,Atlanta Hawks,101.0,102.0
3,2016-04-19,Boston Celtics,Atlanta Hawks,72.0,89.0
4,2016-04-26,Boston Celtics,Atlanta Hawks,83.0,110.0


In [94]:
# Find rows that contain null values
scores_df.isnull().sum()

date               316
visitor              0
home                 0
visitor_points    1446
home_points       1446
dtype: int64

In [95]:
# Drop all rows that have null values
df1 = scores_df.dropna()

# Display
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12796 entries, 0 to 14240
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            12796 non-null  object 
 1   visitor         12796 non-null  object 
 2   home            12796 non-null  object 
 3   visitor_points  12796 non-null  float64
 4   home_points     12796 non-null  float64
dtypes: float64(2), object(3)
memory usage: 599.8+ KB


# Data From Kaggle --> Excel File

In [96]:
df2 = pd.read_excel('./data/games_scores_2019-2022.xlsx')
df2

,date,home_full_name,home_points,visitor_full_name,visitor_points
0,2022-12-22,New Orleans Pelicans,126,San Antonio Spurs,117
1,2022-12-22,Utah Jazz,120,Washington Wizards,112
2,2022-12-21,Cleveland Cavaliers,114,Milwaukee Bucks,106
3,2022-12-21,Philadelphia 76ers,113,Detroit Pistons,93
4,2022-12-21,Atlanta Hawks,108,Chicago Bulls,110
...,...,...,...,...,...
5180,2019-01-01,Toronto Raptors,122,Utah Jazz,116
5181,2019-01-01,Milwaukee Bucks,121,Detroit Pistons,98
5182,2019-01-01,Denver Nuggets,115,New York Knicks,108
5183,2019-01-01,Sacramento Kings,108,Portland Trail Blazers,113


In [97]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5185 entries, 0 to 5184
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   date               5185 non-null   object
 1   home_full_name     5185 non-null   object
 2   home_points        5185 non-null   int64 
 3   visitor_full_name  5185 non-null   object
 4   visitor_points     5185 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 202.7+ KB


In [98]:
# Reference df1 headers
df1.columns.values

array(['date', 'visitor', 'home', 'visitor_points', 'home_points'],
      dtype=object)

In [99]:
# Rename columns to match df1 dataframe
df2 = df2.rename(columns={'visitor_full_name': 'visitor',
                    'home_full_name': 'home'})

# Rearrange columns
df2 = df2[['date', 'visitor', 'home', 'visitor_points', 'home_points']]

# Display
df2.head()


,date,visitor,home,visitor_points,home_points
0,2022-12-22,San Antonio Spurs,New Orleans Pelicans,117,126
1,2022-12-22,Washington Wizards,Utah Jazz,112,120
2,2022-12-21,Milwaukee Bucks,Cleveland Cavaliers,106,114
3,2022-12-21,Detroit Pistons,Philadelphia 76ers,93,113
4,2022-12-21,Chicago Bulls,Atlanta Hawks,110,108


In [100]:
# Union the two Dataframes
final_df = pd.concat([df1, df2], ignore_index=True)

# Sort by date and reset index
final_df = final_df.sort_values(by='date').reset_index(drop=True)

print(f'Total rows from final dataframe: {len(final_df)}')

# Display
final_df

Total rows from final dataframe: 17981


,date,visitor,home,visitor_points,home_points
0,2015-10-02,Denver Nuggets,LA Clippers,96.0,103.0
1,2015-10-03,Charlotte Hornets,Orlando Magic,106.0,100.0
2,2015-10-03,New Orleans Pelicans,Indiana Pacers,110.0,105.0
3,2015-10-04,Utah Jazz,Los Angeles Lakers,90.0,71.0
4,2015-10-04,Charlotte Hornets,Miami Heat,90.0,77.0
...,...,...,...,...,...
17976,2024-11-04,Portland Trail Blazers,New Orleans Pelicans,118.0,100.0
17977,2024-11-04,New York Knicks,Houston Rockets,97.0,109.0
17978,2024-11-04,Memphis Grizzlies,Brooklyn Nets,104.0,106.0
17979,2024-11-04,Utah Jazz,Chicago Bulls,135.0,126.0


In [101]:
# Find rows that are duplicated from the final dataframe
duplicates = final_df[final_df.duplicated()]

print(f'Total duplicates found after union: {len(duplicates)}')
duplicates

Total duplicates found after union: 4823


,date,visitor,home,visitor_points,home_points
4990,2019-01-01,New York Knicks,Denver Nuggets,108.0,115.0
4992,2019-01-01,Detroit Pistons,Milwaukee Bucks,98.0,121.0
4993,2019-01-01,Utah Jazz,Toronto Raptors,116.0,122.0
4994,2019-01-01,Portland Trail Blazers,Sacramento Kings,113.0,108.0
4998,2019-01-02,Minnesota Timberwolves,Boston Celtics,102.0,115.0
...,...,...,...,...,...
15570,2022-12-21,Orlando Magic,Houston Rockets,116.0,110.0
15572,2022-12-21,Golden State Warriors,Brooklyn Nets,113.0,143.0
15573,2022-12-21,Los Angeles Lakers,Sacramento Kings,120.0,134.0
15576,2022-12-22,San Antonio Spurs,New Orleans Pelicans,117.0,126.0


In [102]:
# Find Example
final_df.loc[(final_df['visitor'] == 'Washington Wizards') & (final_df['home'] == 'Utah Jazz')]

,date,visitor,home,visitor_points,home_points
1062,2016-03-11,Washington Wizards,Utah Jazz,93.0,114.0
2639,2017-03-31,Washington Wizards,Utah Jazz,88.0,95.0
3309,2017-12-04,Washington Wizards,Utah Jazz,69.0,116.0
6148,2019-03-29,Washington Wizards,Utah Jazz,124.0,128.0
6157,2019-03-29,Washington Wizards,Utah Jazz,124.0,128.0
8479,2020-02-28,Washington Wizards,Utah Jazz,119.0,129.0
8487,2020-02-28,Washington Wizards,Utah Jazz,119.0,129.0
10799,2021-04-12,Washington Wizards,Utah Jazz,125.0,121.0
10801,2021-04-12,Washington Wizards,Utah Jazz,125.0,121.0
12640,2021-12-18,Washington Wizards,Utah Jazz,109.0,103.0


In [103]:
# Drop duplicates
final_df = final_df.drop_duplicates()

print(f'Total rows of data after dropping duplicates: {len(final_df)}')

Total rows of data after dropping duplicates: 13158


In [104]:
# # There are some rows in the dataframe where all columns except the 'date' column are the same. The difference between these columns is that the dates are off by one day
# # These rows will be removed while only retaining the most recent date 

# # Drop duplicates based on all columns except 'date', keeping the first occurrence
# final_df = final_df.drop_duplicates(subset=['home', 'visitor', 'home_points', 'visitor_points'], keep='first')

# print(f'Final length of data frame after remove rows where dates are off by one day: {len(final_df)}')
# final_df.reset_index(drop=True)

In [105]:
# Export to excel file
final_df.to_excel('./data/gameScores_2015-2024.xlsx', index=False)

# Betting Analysis

In [106]:
# def possible_outcome(dataframe, away_team, home_team, first_line): 
#     # Filter the dataframe for the specific matchup
#     matchup_game = dataframe.loc[(dataframe['visitor'] == away_team) & (dataframe['home'] == home_team)].copy()
    
#     # Add a 'total_points' column 
#     matchup_game['total_points'] = matchup_game['visitor_points'] + matchup_game['home_points']
#     #
#     # Determine the winner
#     matchup_game['winner'] = matchup_game.apply(lambda row: home_team if row['home_points'] > row['visitor_points']  else away_team, axis=1)
    
#     # Create side-by-side plots
#     fig, axes = plt.subplots(1, 2, figsize=(14, 6))
      
#     # Plot total points distribution with a KDE
#     sns.histplot(data=matchup_game, x='total_points', kde=True, ax=axes[0], color="skyblue")
#     axes[0].set_title(f'Total Points Distribution for {away_team} vs {home_team}')
    
#     # Plot the winner count    
#     sns.countplot(data=matchup_game, x='winner', ax=axes[1], hue='winner')
#     axes[1].set_title(f'Winner Counts: {away_team} vs {home_team}')
        
#     # Display the plots
#     plt.show()
    
#      # Calculate a z-score from a provided mean and standard deviation
#     mean = matchup_game['total_points'].mean()
#     standard_deviation = matchup_game['total_points'].std()
   
#     # Create array of UNDER/OVER values starting with given first line
#     under_over_lines = [first_line + i for i in range(15)]
    
#     # Display basic statics of team matchup
#     print(f'Statistics of {away_team} vs {home_team}')
#     print('\n')
#     print(f'Mean: {mean}')
#     print(f'Standard Deviation: {standard_deviation}')

#     print('------------------------------------')
#     # Display the z-scores for each line score
#     for line in under_over_lines:
#         zscore = statistics.NormalDist(mean, standard_deviation).zscore(line)
#         print(f'z-score for {line}: {zscore:.2f}')

#     # print('------------------------------------')

#     # Find percentage of games between specified range
#     # total_games = len(matchup_game)
#     # games_in_range = matchup_game[(matchup_game['total_points'] >= lower_bound) & (matchup_game['total_points'] <= upper_bound)]
#     # percentage = (len(games_in_range) / total_games) * 100
#     # print(f"Percentage of games with total scores between {lower_bound} and {upper_bound}: {percentage:.2f}%")
#     # print('------------------------------------')
    
#     return matchup_game

# away_team = input("Enter the away team: ")
# home_team = input("Enter the home team: ")
# first_line = float(input("Enter the first line for the over/under bet: "))


# possible_outcome(final_df, away_team, home_team, first_line)

In [107]:
# teams = final_df['visitor'].unique()
# teams

# Future Games

In [108]:
# # Get the current date
# current_date = dt.date.today()
# current_date_string = current_date.strftime("%Y-%m-%d")

# # Find games that havent happened yet
# future_games = scores_df[(scores_df['date'] >= current_date_string) & (pd.isna(scores_df['visitor_points']))]
# future_games = future_games.sort_values(by='date')
# future_games